<a href="https://colab.research.google.com/github/jillnoorily/SAandTM_twitterscrapePlayboyMagazine/blob/main/Noorily_tm_gensim_tweets_20221204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDA Topic Modeling with Gensim

Heavily modified and expanded based upon Gensim tutorial

By Jon Chun
28 Mar 2022 Updated

# Setup and Configuration

In [1]:
!pip install python-slugify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=fb133f13ae8b00d1c4dd8384dc69765256ca59017a6216799f07345606040679
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=419ae956ee8929d60228818ad78b00c0af5fe8ba5a7a26d9f1ffd78543c54335
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


In [3]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 5.0 MB/s 
     |████████████████████████████████| 110 kB 53.2 MB/s 


## Configure Jupyter Notebook

In [4]:
## Configure Jupyter Notebook

# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

# Enable multiple outputs from one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
from IPython.display import Image
from ipywidgets import widgets, interactive

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Import Libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
from slugify import slugify

# Get Text Data


## Upload Raw Textfile

In [7]:
from google.colab import files

TEXT_ENCODING = 'utf-8'

In [8]:
%%time

# NOTE: 1m07s

# Upload Plain Text File
uploaded = files.upload()

# NOTE: Allows for multiple file uploads, will only process the last
#       Left in for future feature addition (processing multiple files at once)
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  text_filename_str = fn

# Extract from Dict and decode binary into char string
text_raw_str = uploaded[text_filename_str].decode(TEXT_ENCODING)

Saving tweets_sentiment_playboy-magazine_20200528_122313_20221214_020757 (1).csv to tweets_sentiment_playboy-magazine_20200528_122313_20221214_020757 (1).csv
User uploaded file "tweets_sentiment_playboy-magazine_20200528_122313_20221214_020757 (1).csv" with length 20796619 bytes
CPU times: user 1.78 s, sys: 283 ms, total: 2.07 s
Wall time: 55.7 s


In [9]:
# Recover parameters of original search from filename components
word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')
end_time = ''.join(end_time.split('.')[:-1])

print(f'word1: {word1}')
print(f'word2: {word2}')
print(f'search_str: {search_str}')
print(f'start_date: {start_date}')
print(f'start_time: {start_time}')
print(f'end_date: {end_date}')
print(f'end_time: {end_time}')

word1: tweets
word2: sentiment
search_str: playboy-magazine
start_date: 20200528
start_time: 122313
end_date: 20221214
end_time: 020757 (1)


In [10]:
# Read raw tweet datafile into DataFrame 'data'

try:
  headers = ['datetime','id','tweet','username','reply_ct','retweet_ct','like_ct','quote_ct','retweeted_id','quoted_id',
             'text_raw','text_noemoji','text_clean','vader','textblob','distilbert'] # ,'roberta']
  dtypes = {'datetime':'str',
            'id':'int',
            'tweet':'string',
            'username':'string',
            'reply_ct':'int',
            'retweet_ct':'int',
            'like_ct':'int',
            'quote_ct':'int',
            'retweeted_id':'string',
            'quoted_id':'string',
            'text_raw':'string',
            'text_noemoji':'string',
            'text_clean':'string',
            'vader':'float',
            'textblob':'float',
            'distilbert':'float'}
            # 'roberta':'float'}
  parse_dates = ['datetime']
  # text_df = pd.read_csv(text_filename_str, skiprows=[0], names=headers, dtype=dtypes, parse_dates=parse_dates, index_col=None)
  text_df = pd.read_csv(text_filename_str, index_col=None)
  text_df.head()
  text_df.info()
except:
  print(f'ERROR: Tweets datafile cannot be read by pd.read_csv')

,datetime,id,tweet,username,reply_ct,retweet_ct,like_ct,quote_ct,retweeted_id,quoted_id,text_raw,text_noemoji,text_clean,vader,textblob,distilbert,roberta_twitter_noemoji
0,2020-05-28 12:23:13+00:00,1265981952865128448,"@agoldenmyth Back in 2004, Stone said this in ...",ladyredlpclssic,1,0,1,0,NaN,NaN,"@agoldenmyth Back in 2004, Stone said this in ...","@agoldenmyth Back in 2004 , Stone said this in...",stone say playboy magazine interview interview...,-0.5859,-0.277778,-0.996478,-1
1,2020-05-28 13:03:52+00:00,1265992183737434113,@ariannahuff @AgapiSays I saw your television ...,Irvin3Elligan,0,0,0,0,NaN,NaN,@ariannahuff @AgapiSays I saw your television ...,@ariannahuff @AgapiSays I saw your television ...,see television interview day ago like kitchen ...,0.5859,0.187500,-0.975699,0
2,2020-05-28 13:15:03+00:00,1265994997599436803,@SMH_Historians PhD from @TexasTech 2018. Stud...,bermonkey1096,2,0,6,0,NaN,NaN,@SMH_Historians PhD from @TexasTech 2018. Stud...,@SMH Historians PhD from @TexasTech 2018 . Stu...,historians phd study gender military history c...,0.6808,0.036667,0.821736,0
3,2020-05-28 13:21:17+00:00,1265996567821127682,@ScanlanWithAnA CIA training manuals used to r...,AlangFrize,0,0,1,0,NaN,NaN,@ScanlanWithAnA CIA training manuals used to r...,@ScanlanWithAnA CIA training manuals used to r...,cia training manual recommend magazine supply ...,0.8016,0.420000,0.996525,0
4,2020-05-28 14:26:19+00:00,1266012934267973632,📷 emma-adams: Dinah Willis Playboy Magazine’s ...,edizionimorbose,0,0,0,0,NaN,NaN,📷 emma-adams: Dinah Willis Playboy Magazine’s ...,camera emma-adams Dinah Willis Playboy Magazin...,camera emma adam dinah willis playboy magazine...,0.0000,0.000000,-0.966918,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   datetime                 30000 non-null  object 
 1   id                       30000 non-null  int64  
 2   tweet                    30000 non-null  object 
 3   username                 30000 non-null  object 
 4   reply_ct                 30000 non-null  int64  
 5   retweet_ct               30000 non-null  int64  
 6   like_ct                  30000 non-null  int64  
 7   quote_ct                 30000 non-null  int64  
 8   retweeted_id             0 non-null      float64
 9   quoted_id                1703 non-null   object 
 10  text_raw                 30000 non-null  object 
 11  text_noemoji             30000 non-null  object 
 12  text_clean               30000 non-null  object 
 13  vader                    30000 non-null  float64
 14  textblob              

## Clean DataFrame

In [11]:
# Drop unused columns

text_df.drop(columns=['reply_ct','retweet_ct','like_ct','quote_ct','retweeted_id','quoted_id'], inplace=True)
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   datetime                 30000 non-null  object 
 1   id                       30000 non-null  int64  
 2   tweet                    30000 non-null  object 
 3   username                 30000 non-null  object 
 4   text_raw                 30000 non-null  object 
 5   text_noemoji             30000 non-null  object 
 6   text_clean               30000 non-null  object 
 7   vader                    30000 non-null  float64
 8   textblob                 30000 non-null  float64
 9   distilbert               30000 non-null  float64
 10  roberta_twitter_noemoji  30000 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 2.5+ MB


In [12]:
# Clean and drop illegal datetime values

text_df['datetime'] = pd.to_datetime(text_df['datetime'], errors='coerce')
text_df = text_df.dropna(subset=['datetime'])
text_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   datetime                 30000 non-null  datetime64[ns, UTC]
 1   id                       30000 non-null  int64              
 2   tweet                    30000 non-null  object             
 3   username                 30000 non-null  object             
 4   text_raw                 30000 non-null  object             
 5   text_noemoji             30000 non-null  object             
 6   text_clean               30000 non-null  object             
 7   vader                    30000 non-null  float64            
 8   textblob                 30000 non-null  float64            
 9   distilbert               30000 non-null  float64            
 10  roberta_twitter_noemoji  30000 non-null  int64              
dtypes: datetime64[ns, UTC](1), f

In [13]:
# Drop any row that have at any NaN

text_df.dropna(inplace=True)
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   datetime                 30000 non-null  datetime64[ns, UTC]
 1   id                       30000 non-null  int64              
 2   tweet                    30000 non-null  object             
 3   username                 30000 non-null  object             
 4   text_raw                 30000 non-null  object             
 5   text_noemoji             30000 non-null  object             
 6   text_clean               30000 non-null  object             
 7   vader                    30000 non-null  float64            
 8   textblob                 30000 non-null  float64            
 9   distilbert               30000 non-null  float64            
 10  roberta_twitter_noemoji  30000 non-null  int64              
dtypes: datetime64[ns, UTC](1), f

In [14]:
text_df['text_raw'] = text_df['text_raw']

text_df['text_raw_type'] = text_df['text_raw'].apply(lambda x: type(x))
text_df['text_raw_type'].value_counts()

<class 'str'>    30000
Name: text_raw_type, dtype: int64

In [15]:
text_df.convert_dtypes()
text_df.info()

,datetime,id,tweet,username,text_raw,text_noemoji,text_clean,vader,textblob,distilbert,roberta_twitter_noemoji,text_raw_type
0,2020-05-28 12:23:13+00:00,1265981952865128448,"@agoldenmyth Back in 2004, Stone said this in ...",ladyredlpclssic,"@agoldenmyth Back in 2004, Stone said this in ...","@agoldenmyth Back in 2004 , Stone said this in...",stone say playboy magazine interview interview...,-0.5859,-0.277778,-0.996478,-1,<class 'str'>
1,2020-05-28 13:03:52+00:00,1265992183737434113,@ariannahuff @AgapiSays I saw your television ...,Irvin3Elligan,@ariannahuff @AgapiSays I saw your television ...,@ariannahuff @AgapiSays I saw your television ...,see television interview day ago like kitchen ...,0.5859,0.1875,-0.975699,0,<class 'str'>
2,2020-05-28 13:15:03+00:00,1265994997599436803,@SMH_Historians PhD from @TexasTech 2018. Stud...,bermonkey1096,@SMH_Historians PhD from @TexasTech 2018. Stud...,@SMH Historians PhD from @TexasTech 2018 . Stu...,historians phd study gender military history c...,0.6808,0.036667,0.821736,0,<class 'str'>
3,2020-05-28 13:21:17+00:00,1265996567821127682,@ScanlanWithAnA CIA training manuals used to r...,AlangFrize,@ScanlanWithAnA CIA training manuals used to r...,@ScanlanWithAnA CIA training manuals used to r...,cia training manual recommend magazine supply ...,0.8016,0.42,0.996525,0,<class 'str'>
4,2020-05-28 14:26:19+00:00,1266012934267973632,📷 emma-adams: Dinah Willis Playboy Magazine’s ...,edizionimorbose,📷 emma-adams: Dinah Willis Playboy Magazine’s ...,camera emma-adams Dinah Willis Playboy Magazin...,camera emma adam dinah willis playboy magazine...,0.0,0.0,-0.966918,0,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2022-12-14 00:13:27+00:00,1602819018640998401,@What_the_Cheick @againstgrmrs Here’s somethin...,TheMikeyChill,@What_the_Cheick @againstgrmrs Here’s somethin...,@What the Cheick @againstgrmrs Here ' s someth...,cheick s appropriate playboy hustler penthouse...,-0.5423,0.066667,-0.998616,-1,<class 'str'>
29996,2022-12-14 00:23:06+00:00,1602821445004005377,@Ambers_Couch @SteelySilver @Biddyone11 @FeelF...,bunnybmbshell,@Ambers_Couch @SteelySilver @Biddyone11 @FeelF...,@Ambers Couch @SteelySilver @Biddyone11 @FeelF...,couch n plus playboy magazine reality fox call...,-0.0516,-0.094444,-0.97003,0,<class 'str'>
29997,2022-12-14 00:53:49+00:00,1602829177866043393,@_Peace_Love_US Maybe Playboy Magazine but not...,papettyconcan,@_Peace_Love_US Maybe Playboy Magazine but not...,@ Peace Love US Maybe Playboy Magazine but not...,peace love maybe playboy magazine prison,0.6597,0.5,-0.845957,0,<class 'str'>
29998,2022-12-14 01:15:41+00:00,1602834680214167553,La copine transsexuelle du footballeur françai...,wisly_promo,La copine transsexuelle du footballeur françai...,La copine transsexuelle du footballeur francai...,la copine transsexuelle du footballeur francai...,0.0,0.0,-0.89586,0,<class 'str'>


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   datetime                 30000 non-null  datetime64[ns, UTC]
 1   id                       30000 non-null  int64              
 2   tweet                    30000 non-null  object             
 3   username                 30000 non-null  object             
 4   text_raw                 30000 non-null  object             
 5   text_noemoji             30000 non-null  object             
 6   text_clean               30000 non-null  object             
 7   vader                    30000 non-null  float64            
 8   textblob                 30000 non-null  float64            
 9   distilbert               30000 non-null  float64            
 10  roberta_twitter_noemoji  30000 non-null  int64              
 11  text_raw_type            300

In [16]:
# Try to convert Tweet text to string (raise error if incompatible encoding)

# text_df['text_raw'] = text_df['text_raw'].apply(lambda x: str(x)) # 
text_df[['text_raw','username','tweet','text_noemoji','text_clean']] = text_df[['text_raw','username','tweet','text_noemoji','text_clean']].astype('string', errors='raise') # , errors='coerce')
# text_df = text_df.dropna(subset=['text_raw'])
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   datetime                 30000 non-null  datetime64[ns, UTC]
 1   id                       30000 non-null  int64              
 2   tweet                    30000 non-null  string             
 3   username                 30000 non-null  string             
 4   text_raw                 30000 non-null  string             
 5   text_noemoji             30000 non-null  string             
 6   text_clean               30000 non-null  string             
 7   vader                    30000 non-null  float64            
 8   textblob                 30000 non-null  float64            
 9   distilbert               30000 non-null  float64            
 10  roberta_twitter_noemoji  30000 non-null  int64              
 11  text_raw_type            300

In [17]:
line_ls = text_df.text_clean.to_list()
print(f'BEFORE filtering out NAType: len(line_ls): {len(line_ls)}')

line_ls = [x for x in line_ls if isinstance(x, str)]
print(f' AFTER filtering out NAType: len(line_ls): {len(line_ls)}')

line_ls = [x for x in line_ls if len(x) > 0]
print(f' AFTER filtering out null lines: len(line_ls): {len(line_ls)}')

BEFORE filtering out NAType: len(line_ls): 30000
 AFTER filtering out NAType: len(line_ls): 30000
 AFTER filtering out null lines: len(line_ls): 30000


In [18]:
# Agglomerate n Tweets into a Synthetic Document for Topic Modeling

doc_ls = []
tweets_per_doc = 10
doc_current_ls = []
j = 0

for i, aline in enumerate(line_ls):
  
  if (((i % tweets_per_doc) == 0) & (i>0)):
    doc_temp_str = '\n'.join(doc_current_ls)
    doc_ls.append(doc_temp_str)
    print(f'Synthetic Doc #{j}: {doc_temp_str}\n\n')
    j += 1
    # Reset current syn doc
    doc_current_ls = []
    doc_current_ls.append(aline)

  else:
    try:
      doc_current_ls.append(aline)
    except:
      print(f'ERROR: Cannot strip()')

  # line_doc_ls.append(aline)

print('\n\n')

text_raw_str = "\n\n".join(doc_ls)
text_raw_str[:5]


Streaming output truncated to the last 5000 lines.
playboy carti sue chemical romance profit fan rookie magazine describe result artistically drive inspire
sister catfish people aol like time hold playboy magazine web cam accidentally drop face tear joy face tear joy face tear joy face tear joy face tear joy face tear joy
husband look playboy magazine bathroom


Synthetic Doc #2583: disappoint long explicit nudity magazine feel like openness human body well like kid go instead dressup stuff kinky garbage root
remind miyamotto want announce super mario playboy magazine flush face url
playboy transexual man cover wonder long magazine print know hugh roll grave smh face rolling eye
d tell touch picture female playboy magazine ask like say yeah sense let house sense voluntarily hand home
playboy magazine
want feel princess play girl movie modeling agency queen princess empire magazine playboy black bunny book
good morning west playboy remove african water defence magazine wdm url
playboy m

'stone'

In [19]:
doc_ls[-1]
len(doc_ls)

'playboy magazine december denise richards vintage adult magazine screaminggreek playboy deniserichard hey malakas check url cool crap url url\nugly people baz lurhman hugh jackman playboy magazine s ugly kid joe toothless joe pedestrian ex guy\nrumor stop revive november playboy magazine article title bed elvis write byron raphael alanna nash article byron claim work elvis url\nauction penthouse magazine s lot url ebay penthousemagazine seventy penthouselot auction bidnow playboy vintagepenthouse mensmag gift giftidea penthousecollector giftforhim christmasgift mangift bidnow\ndovkleiman hate leer bunch playboy magazine shoot\ndidyouknow christmas story base short story publish magazine christmasstory achristmasstory christmastrivia christmas url\nhugh hefner month away publish issue playboy magazine bear\nmindset sure want school librarie stock playboy magazine example sure agree limit kid\nplayboy festeggia il suo sessantanovesimo compleanno su sandbox leggi larticolo cliccando qui 

2999

# Enter Title

In [20]:
# If this is a Twitter textfile, the search terms are recovered from the filename

# Recover parameters of original search from filename components
try:
  word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')

  print(f'word1: {word1}')
  print(f'word2: {word2}')
  print(f'search_str: {search_str}')
  print(f'start_date: {start_date}')
  print(f'end_date: {end_date}')
  print('\n')
  
  text_title_str = f'[{search_str}] from [{start_date}] to [{end_date}]'
  print(f'\nTwitter Search String:\n\n  {text_title_str}\n')

except:
  print('This is not a Twitter datafile named in standard format\n\nEnter the Novel information below.')

word1: tweets
word2: sentiment
search_str: playboy-magazine
start_date: 20200528
end_date: 20221214



Twitter Search String:

  [playboy-magazine] from [20200528] to [20221214]



In [21]:
#@title Enter Title "[Title] by [Author]""

Text_Title = "Playboy Magazine on Twitter from 2020 to 2022" #@param {type:"string"}

# Remove illegal filename punctuation
Text_Title = Text_Title.replace("'","")
Text_Title = Text_Title.replace('"','')
Text_Title = Text_Title.replace('.',' ')

text_title_str = ' '.join([x.capitalize() for x in slugify(Text_Title).replace('-',' ').split()])
text_title_str

'Playboy Magazine On Twitter From 2020 To 2022'


# LDA Model

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


The purpose of this tutorial is to demonstrate how to train and tune an LDA model.

In this tutorial we will:

* Load input data.
* Pre-process that data.
* Transform documents into bag-of-words vectors.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you all the parameters and options for Gensim's LDA implementation

If you are not familiar with the LDA model or how to use it in Gensim, I (Olavur Mortensen)
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.





## Segment Text

In [22]:
text_raw_str[:500]
print('\n')
len(text_raw_str)

'stone say playboy magazine interview interview url kiss scene leave cutting room floor seriously wb cut scene jared film loudly cry face url\nsee television interview day ago like kitchen table fireside chat camera window tv studio author piccaso original follow twitter interview playboy magazine greek\nhistorians phd study gender military history current project manuscript playboy magazine role vietnam significance soldier future interest include pop culture warfare comicstrip pornography social '

2801098

In [23]:
"""
# Split into paragraphs
text_parags_ls = text_raw_str.split('\n')
print(f'There are {len(text_parags_ls)} raw paragraphs\n')

# Delete empty paragraphs
text_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]
print(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')

# Replace stray/embedded /n with a space
text_parags_clean_ls = []
for aparag in text_parags_ls:
  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())

text_parags_ls = text_parags_clean_ls
text_parags_ls[:5] 
"""

text_parags_ls = doc_ls

"\n# Split into paragraphs\ntext_parags_ls = text_raw_str.split('\n')\nprint(f'There are {len(text_parags_ls)} raw paragraphs\n')\n\n# Delete empty paragraphs\ntext_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]\nprint(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')\n\n# Replace stray/embedded /n with a space\ntext_parags_clean_ls = []\nfor aparag in text_parags_ls:\n  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())\n\ntext_parags_ls = text_parags_clean_ls\ntext_parags_ls[:5] \n"

In [24]:
# Verify first paragraph (after title and chapter headings)

text_parags_ls[3]
print('\n')
# Paragraph char count
len(text_parags_ls[3])

'playboy magazine m porn star url\nwrite playboy magazine publish september write ruben whittington pottstown pennsylvania blacklivesmatter url\ntowel well prop bead rope look get wade pool hot tub bareness overly face look young man magazine mag little year\nknow photoshoot right c actually hugh hefner inspiration famous adult magazine playboy url\nthrow couple playboy magazine familiarize monster derail path success slightly smile face\nplayboy magazine layoffs closure print magazine coronavirus effects wwd url\nread article playboy ve read article magazine\ns jail rank playboy magazine father law work\nremember european playstation magazine fuck playboy like couple issue porn ad plaster page\nneed day normal man bathroom check new playboy magazine shit'

755

In [25]:
# Delete any paragraphs shorter than MIN_LEN_PARAG

MIN_LEN_PARAG = 5
MIN_LEN_DOC = 100

# Delete any paragraphs shorter than MIN_LEN_PARAG
text_parags_ls = [x for x in text_parags_ls if len(x) > MIN_LEN_PARAG]

# Trim any leading/trailing/multiple embedded whitespaces
text_parags_ls = [' '.join(x.split()) for x in text_parags_ls]

len(text_parags_ls)

2999

In [26]:
# Agglomerate paragraphs into Documents of MIN_LEN_DOC=1000 chars
"""
parag_ct = len(text_parags_ls)

doc_now_str = ''
doc_now_len = 0
docs = []

for i in range(parag_ct):
  # print(f'Processing Paragraph #{i}')
  parag_now_str = text_parags_ls[i]
  doc_now_str += parag_now_str
  doc_now_len += len(parag_now_str)
  if doc_now_len > MIN_LEN_DOC:
    docs.append(doc_now_str)
    doc_now_str = ''
    doc_now_len = 0

docs[-1] += doc_now_str

print(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')
"""

"\nparag_ct = len(text_parags_ls)\n\ndoc_now_str = ''\ndoc_now_len = 0\ndocs = []\n\nfor i in range(parag_ct):\n  # print(f'Processing Paragraph #{i}')\n  parag_now_str = text_parags_ls[i]\n  doc_now_str += parag_now_str\n  doc_now_len += len(parag_now_str)\n  if doc_now_len > MIN_LEN_DOC:\n    docs.append(doc_now_str)\n    doc_now_str = ''\n    doc_now_len = 0\n\ndocs[-1] += doc_now_str\n\nprint(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')\n"

In [27]:
# View the first 5 docs

doc_ls[:5]

['stone say playboy magazine interview interview url kiss scene leave cutting room floor seriously wb cut scene jared film loudly cry face url\nsee television interview day ago like kitchen table fireside chat camera window tv studio author piccaso original follow twitter interview playboy magazine greek\nhistorians phd study gender military history current project manuscript playboy magazine role vietnam significance soldier future interest include pop culture warfare comicstrip pornography social medium interested soldier cope\ncia training manual recommend magazine supply list good one playboy highly recommend bbc cookery magazine woman weekly penthouse available\ncamera emma adam dinah willis playboy magazine s playmate month december url\nurl j trump sr president united states ex wife karen mcdougal playboy magazine mysterious model melania trump fist lady white house oval office washington dc capital hill donald j trump sr wife\nplayboy carti announce new music video time magazin

In [28]:
# View the first 500 chars in the 50th doc

doc_ls[49][:500]

'dear blow dust magazine year old copy playboy url\nthing discover interesting see post online yanno pick copy face tear joy face tear joy jellyroll playboy magazine pick copy sure url\nsmirk face oho yes playboy rabbit magazine\nstudent pour tiananmen square chinese government blow trump tell playboy magazine month later vicious horrible strength show power strength url\ndonald trump interview playboy magazine hundred thousand student massacre govt tell need know doesn t make week surprising url\nser'

In [29]:
# View the last 100 chars in the last doc

doc_ls[-1][-100:]

'ayboy magazine pack big league chew gum pat bev kd str russ joel embid james harden face rolling eye'

## Pre-process and vectorize the documents

As part of preprocessing, we will:

* Expand Contractions
* Tokenize (split the documents into tokens)
* Define stopwords and filter out
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




### Steps

#### Expand Contractions

In [30]:
# Expand Contractions (e.g. can't -> can not)

import contractions

# Test
contractions.fix("yall're happy now it isn't a don't tell problem", slang=False) # default: true

"yall're happy now it is not a do not tell problem"

In [31]:
# Expand all Contractions paragraph by paragraph
docs_clean_ls = []
for adoc in doc_ls:
  docs_clean_ls.append(contractions.fix(adoc))

docs = docs_clean_ls
docs[:5]

['stone say playboy magazine interview interview url kiss scene leave cutting room floor seriously wb cut scene jared film loudly cry face url\nsee television interview day ago like kitchen table fireside chat camera window tv studio author piccaso original follow twitter interview playboy magazine greek\nhistorians phd study gender military history current project manuscript playboy magazine role vietnam significance soldier future interest include pop culture warfare comicstrip pornography social medium interested soldier cope\ncia training manual recommend magazine supply list good one playboy highly recommend bbc cookery magazine woman weekly penthouse available\ncamera emma adam dinah willis playboy magazine s playmate month december url\nurl j trump sr president united states ex wife karen mcdougal playboy magazine mysterious model melania trump fist lady white house oval office washington dc capital hill donald j trump sr wife\nplayboy carti announce new music video time magazin

#### Tokenize Text

In [32]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [33]:
# Verify docs

# content and token count of first doc
docs[0]
print(f'\n\nThere are {len(docs[0])} tokens in the first document')

['stone',
 'say',
 'playboy',
 'magazine',
 'interview',
 'interview',
 'url',
 'kiss',
 'scene',
 'leave',
 'cutting',
 'room',
 'floor',
 'seriously',
 'wb',
 'cut',
 'scene',
 'jared',
 'film',
 'loudly',
 'cry',
 'face',
 'url',
 'see',
 'television',
 'interview',
 'day',
 'ago',
 'like',
 'kitchen',
 'table',
 'fireside',
 'chat',
 'camera',
 'window',
 'tv',
 'studio',
 'author',
 'piccaso',
 'original',
 'follow',
 'twitter',
 'interview',
 'playboy',
 'magazine',
 'greek',
 'historians',
 'phd',
 'study',
 'gender',
 'military',
 'history',
 'current',
 'project',
 'manuscript',
 'playboy',
 'magazine',
 'role',
 'vietnam',
 'significance',
 'soldier',
 'future',
 'interest',
 'include',
 'pop',
 'culture',
 'warfare',
 'comicstrip',
 'pornography',
 'social',
 'medium',
 'interested',
 'soldier',
 'cope',
 'cia',
 'training',
 'manual',
 'recommend',
 'magazine',
 'supply',
 'list',
 'good',
 'one',
 'playboy',
 'highly',
 'recommend',
 'bbc',
 'cookery',
 'magazine',
 'woman



There are 175 tokens in the first document


#### Customize Stopwords

In [34]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_ls = stopwords.words('english')

print(f'\nThe first ten stopwords:')
stopwords_ls[20:]
print(f'\n\nThere are [{len(stopwords_ls)}] English stopwords imported from NLTK')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


The first ten stopwords:


['himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don



There are [179] English stopwords imported from NLTK


In [35]:
# Define list of stopwords listed above that we should keep

stopword_keep_ls = []

"""
stopword_keep_ls = [
    'against',
    'not',
    'no',
    'nor',

    'do',
    'are',
    'is',
    'are',
    "not",
    'could',
    'did',
    'have',
    'has',
    'had',
    'might',
    'must',
    'need',
    'shall',
    'should',
    'was',
    'were',
    'would',

    'own',
    'same',
    'how',
    'why'
]
"""

'\nstopword_keep_ls = [\n    \'against\',\n    \'not\',\n    \'no\',\n    \'nor\',\n\n    \'do\',\n    \'are\',\n    \'is\',\n    \'are\',\n    "not",\n    \'could\',\n    \'did\',\n    \'have\',\n    \'has\',\n    \'had\',\n    \'might\',\n    \'must\',\n    \'need\',\n    \'shall\',\n    \'should\',\n    \'was\',\n    \'were\',\n    \'would\',\n\n    \'own\',\n    \'same\',\n    \'how\',\n    \'why\'\n]\n'

In [36]:
# Derive final stopword list

stopword_keep_ls = list(set(stopwords_ls) - set(stopword_keep_ls))
stopword_keep_ls
print('\n\n')
print(f'BEFORE: {len(stopwords_ls)} stopwords')
print(f' AFTER: {len(stopword_keep_ls)}')

['whom',
 'o',
 'down',
 'don',
 'y',
 'and',
 "isn't",
 'has',
 'were',
 'been',
 'other',
 'which',
 'than',
 'once',
 'wasn',
 'off',
 'during',
 'further',
 'yours',
 'own',
 'any',
 'having',
 'theirs',
 'myself',
 'are',
 'these',
 'both',
 'mustn',
 'only',
 'ours',
 'nor',
 "shan't",
 'to',
 've',
 'most',
 'did',
 'be',
 'what',
 'in',
 'how',
 'will',
 'more',
 'ourselves',
 'his',
 "she's",
 'there',
 "mustn't",
 'for',
 'again',
 'our',
 "don't",
 'those',
 't',
 'below',
 'but',
 'didn',
 "aren't",
 'hasn',
 'with',
 "needn't",
 "that'll",
 'them',
 'had',
 'just',
 "it's",
 'they',
 'was',
 'against',
 'am',
 'aren',
 'him',
 'no',
 'won',
 "doesn't",
 'isn',
 'couldn',
 'very',
 'or',
 'i',
 'being',
 'such',
 'by',
 'a',
 'she',
 'so',
 'yourselves',
 'all',
 'have',
 'd',
 'because',
 'up',
 'over',
 'do',
 'does',
 'themselves',
 'now',
 "you're",
 'needn',
 'if',
 "shouldn't",
 "wouldn't",
 's',
 'of',
 'itself',
 'its',
 'why',
 'before',
 'ma',
 'wouldn',
 'her',
 




BEFORE: 179 stopwords
 AFTER: 179


In [37]:
# Remove all stopwords with contractions (optional - could also remove fragments like 'll', 're', and 've' but not necessary)

stopwords_ls = [x for x in stopword_keep_ls if not "'" in x]
stopwords_ls[20:]
print(f'\n\nThere are now {len(stopwords_ls)} stopwords after removing words with contractions')


['having',
 'theirs',
 'myself',
 'are',
 'these',
 'both',
 'mustn',
 'only',
 'ours',
 'nor',
 'to',
 've',
 'most',
 'did',
 'be',
 'what',
 'in',
 'how',
 'will',
 'more',
 'ourselves',
 'his',
 'there',
 'for',
 'again',
 'our',
 'those',
 't',
 'below',
 'but',
 'didn',
 'hasn',
 'with',
 'them',
 'had',
 'just',
 'they',
 'was',
 'against',
 'am',
 'aren',
 'him',
 'no',
 'won',
 'isn',
 'couldn',
 'very',
 'or',
 'i',
 'being',
 'such',
 'by',
 'a',
 'she',
 'so',
 'yourselves',
 'all',
 'have',
 'd',
 'because',
 'up',
 'over',
 'do',
 'does',
 'themselves',
 'now',
 'needn',
 'if',
 's',
 'of',
 'itself',
 'its',
 'why',
 'before',
 'ma',
 'wouldn',
 'her',
 'this',
 'is',
 'doing',
 'you',
 'shouldn',
 'who',
 'when',
 'too',
 'above',
 're',
 'out',
 'through',
 'me',
 'll',
 'doesn',
 'as',
 'here',
 'weren',
 'some',
 'an',
 'until',
 'while',
 'hers',
 'that',
 'ain',
 'the',
 'mightn',
 'each',
 'm',
 'we',
 'into',
 'not',
 'under',
 'he',
 'it',
 'about',
 'should',
 



There are now 153 stopwords after removing words with contractions


**[CUSTOMIZE] Stopwords list by adding or deleting tokens**

In [38]:
# [CUSTOMIZE] Stopwords to ADD or DELETE from default NLTK English stopword list

STOPWORDS_ADD_SET = set(['bazinga', 'woo-hoo']) # Edit this list to add new words to the stopwords list
STOPWORDS_DEL_SET = set(['the']) # Edit this list to remove exising words from the stopwords list

stopwords_en_ls = list(set(stopwords_ls).difference(set(STOPWORDS_DEL_SET)).union(set(STOPWORDS_ADD_SET)))
print(f'Final Count after Customized Add/Del: {len(stopwords_en_ls)} Stopwords')

Final Count after Customized Add/Del: 154 Stopwords


#### Lemmatize

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [39]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [40]:
%%time

# NOTE: 0m24s @03:48 on 20220228 Colab Pro 

# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

CPU times: user 3.36 s, sys: 82.6 ms, total: 3.44 s
Wall time: 3.47 s


We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




### Identify Bi- and Tri-Grams

In [41]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [42]:
#@markdown Minimum number of documents a word must appear in: (default 20)

Min_No_Documents = 5 #@param {type:"slider", min:5, max:100, step:1}

#@markdown Max percent of documents a word can appear in: (default 0.50 or 50%)

Max_Percent_Documents = 0.49 #@param {type:"slider", min:0.05, max:0.9, step:0.01}


In [43]:
# docs_ls_ls = [astring.split() for astring in docs]
# docs = docs_ls_ls

In [44]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=Min_No_Documents, no_above=Max_Percent_Documents)

Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [45]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Let's see how many tokens and documents we have to train on.




In [46]:
# Corpus as a list of documents, each a list of tokens identifyed by dictionary tuples

len(corpus)

2999

In [47]:
type(corpus[0])

list

In [48]:
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 2),
 (9, 1)]

In [49]:
print('Number of unique tokens: %d' % len(dictionary))  # Orig 1864
print('Number of documents: %d' % len(corpus))          # Orig 1740

Number of unique tokens: 6966
Number of documents: 2999


## Training

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [66]:
#@markdown How many Topics do you want to find?

No_of_Topics = 20 #@param {type:"slider", min:2, max:200, step:1}

#@markdown Default 10-50 depending upon how large the text and diverse the vocabulary

In [67]:
%%time

# NOTE: 2m01s @20220328 with Google Colab/CPU on Harry Potter

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = No_of_Topics
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

CPU times: user 54.1 s, sys: 532 ms, total: 54.7 s
Wall time: 54.4 s


We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [68]:
top_topics = model.top_topics(corpus) #, num_words=20)


# from pprint import pprint
# pprint(top_topics)

# print('\n\n')
# for i,atopic in enumerate(top_topics):
#   atopic_coherence_str = f'Topic #{i}: coherence = {atopic[1]}'
#   print(atopic_coherence_str)
#   report_line_ls.append(atopic_coherence_str)


topic_word_ls = []

topic_word_ls.append(text_title_str)

# print(text_title_str)

for i, atopic in enumerate(top_topics):
  topic_str = f'\n\nTopic #{i}: {atopic[1]} (coherence)'
  # print(topic_str)
  topic_word_ls.append(topic_str)
  for j, aword in enumerate(atopic[0]):
    word_str = f'  Word #{j}: {aword}'
    # print(word_str)
    topic_word_ls.append(word_str)

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
avg_topic_coherence_str = f'Average topic coherence: {avg_topic_coherence:.4f}'
# print(avg_topic_coherence_str)
topic_word_ls.append(f'\n\n{avg_topic_coherence_str}')

topic_word_str = "\n".join(topic_word_ls)
print(topic_word_str)

# Save to file
filename_topic_word_str = f'lda_topic_word_report.txt'
with open(filename_topic_word_str, 'w') as fp:
  fp.write(topic_word_str)

files.download(filename_topic_word_str)

Playboy Magazine On Twitter From 2020 To 2022


Topic #0: -1.0517918583497203 (coherence)
  Word #0: (0.059772372, 'tear')
  Word #1: (0.057520945, 'joy')
  Word #2: (0.041356906, 'face_tear')
  Word #3: (0.015375075, 'tear_joy')
  Word #4: (0.014959305, 'joy_face')
  Word #5: (0.0067620683, 'look')
  Word #6: (0.006608308, 'cover')
  Word #7: (0.0061609144, 'man')
  Word #8: (0.0058855494, 'read')
  Word #9: (0.0058237594, 'think')
  Word #10: (0.004744796, 'trump')
  Word #11: (0.0045644357, 'woman')
  Word #12: (0.0044841375, 'old')
  Word #13: (0.0044799526, 'time')
  Word #14: (0.004389708, 'know')
  Word #15: (0.0039435597, 'go')
  Word #16: (0.0039372724, 'get')
  Word #17: (0.00385913, 'boy')
  Word #18: (0.003810234, 'good')
  Word #19: (0.0036806671, 'article')


Topic #1: -1.1286733234098985 (coherence)
  Word #0: (0.010780856, 'old')
  Word #1: (0.009847141, 'find')
  Word #2: (0.008747846, 'year')
  Word #3: (0.008074039, 'know')
  Word #4: (0.007456822, 'look')
  Word #5:

15225

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Visualize

In [69]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [70]:
# vis_data = gensimvis.prepare(lda, corpus, dictionary)
vis_data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis_data)

## Things to experiment with

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

## Where to go from here

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

## References

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


